- Overview of example system with data interfaces
- air source heat pump
- What inputs/information do we need? e.g. COP, demand time series, price electricity, ...
- How can we generate those inputs


![solph graph](./workshop/figures/energy_system_solph.svg)


In [ ]:
from workshop.model.utilities import load_input_data

input_data = load_input_data().head(24*2)

result_dict = {}


In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

es.add(bus_electricity, bus_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

demand = input_data["Heat load (kW)"][:-1]

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


es.add(electricity_grid, thermal_storage, heat_demand)

Example heat pump: https://www.viessmann.co.uk/en/products/heat-pump/vitocal-150a.html (type 150.A16)

In [ ]:
hp_thermal_power = 9.1  # kW
cop_a7_w35 = 4.9

In [ ]:
heat_pump = heat_pump_constant = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        bus_electricity: 1,
        bus_heat_35C: cop_a7_w35,
    },
)


es.add(heat_pump_constant)


model_constant = solph.Model(energysystem=es)
model_constant.solve()


result_dict["constant"] = solph.processing.results(model_constant)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

results = result_dict["constant"]

heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(9, 4.5))

ax1.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
ax1.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
ax1.plot(storage_content, "k-", label="storage content")
ax1.set_ylabel("Power (kW) or Energy (kWh)")
ax1.tick_params(axis="x", rotation=50)
ax1.grid()
ax1.legend()

ax2.plot(np.sort(heat_supply)[-2::-1], "r-", label="heat supply")
ax2.plot(np.sort(demand)[::-1], "b--", label="heat demand")
ax2.plot(np.sort(storage_content)[::-1], "k-", label="storage content")
ax2.grid()
ax2.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")

Temperature-dependent COP

![solph graph](./workshop/figures/heat_pump_blackbox_comparison.svg)

- COP depends on temperature, we only know a single temperature value
- Solution: Use Carnot method with constant efficiency assumption
- Reference temperature for Carnot COP is unclear, 7°C -> 35°C vs. 2°C -> 40°C

In [ ]:
carnot_cop_7_35 = (35+273.15) / (35-7)
cpf_7_35 = cop_a7_w35 / carnot_cop_7_35

carnot_cop_2_40 = (40+273.15) / (40-2)
cpf_2_40 = cop_a7_w35 / carnot_cop_2_40

input_data["cpf COP 7 -> 35"] = cpf_7_35 * (35+273.15) / (35 - input_data["Ambient temperature (°C)"])
input_data["cpf COP 2 -> 40"] = cpf_2_40 * (40+273.15) / (40 - input_data["Ambient temperature (°C)"] + 5)

plt.plot(input_data["Ambient temperature (°C)"], "b-")
plt.ylabel("Ambient temperature (°C)").set_color("blue")


plt.tick_params(axis="x", rotation=50)
plt.twinx()

plt.plot(input_data["cpf COP 7 -> 35"], "r-", label="Carnot: 7 -> 35")
plt.plot(input_data["cpf COP 2 -> 40"], "r--", label="Carnot: 2 -> 40")
plt.ylabel("COP").set_color("red")
plt.legend()
plt.show()

In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

es.add(bus_electricity, bus_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

demand = input_data["Heat load (kW)"][:-1]

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


es.add(electricity_grid, thermal_storage, heat_demand)

In [ ]:

heat_pump = heat_pump_carnot = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        bus_electricity: 1,
        bus_heat_35C: input_data["cpf COP 7 -> 35"][:-1],
    },
)

es.add(heat_pump_carnot)


model_carnot = solph.Model(energysystem=es)
model_carnot.solve()


results = result_dict["carnot"] = solph.processing.results(model_carnot)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(9, 4.5))

ax1.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
ax1.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
ax1.plot(storage_content, "k-", label="storage content")
ax1.set_ylabel("Power (kW) or Energy (kWh)")
ax1.tick_params(axis="x", rotation=50)
ax1.grid()
ax1.legend()

ax2.plot(np.sort(heat_supply)[-2::-1], "r-", label="heat supply")
ax2.plot(np.sort(demand)[::-1], "b--", label="heat demand")
ax2.plot(np.sort(storage_content)[::-1], "k-", label="storage content")
ax2.grid()
ax2.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")

- Then TESPy model to generate COP, reconfigure the heat pump in solph and run again.

In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

es.add(bus_electricity, bus_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

demand = input_data["Heat load (kW)"][:-1]

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


es.add(electricity_grid, thermal_storage, heat_demand)

In [ ]:
from workshop.model.utilities import load_tespy_cop

tespy_cop = load_tespy_cop()

input_data["simple TESPy COP"] = input_data["Ambient temperature (d°C)"].map(tespy_cop["COP"])

heat_pump = heat_pump_tespy = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        bus_electricity: 1,
        bus_heat_35C: input_data["simple TESPy COP"][:-1],
    },
)

es.add(heat_pump_tespy)


model_tespy = solph.Model(energysystem=es)
model_tespy.solve()


results = result_dict["tespy"] = solph.processing.results(model_tespy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(9, 4.5))

ax1.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
ax1.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
ax1.plot(storage_content, "k-", label="storage content")
ax1.set_ylabel("Power (kW) or Energy (kWh)")
ax1.tick_params(axis="x", rotation=50)
ax1.grid()
ax1.legend()

ax2.plot(np.sort(heat_supply)[-2::-1], "r-", label="heat supply")
ax2.plot(np.sort(demand)[::-1], "b--", label="heat demand")
ax2.plot(np.sort(storage_content)[::-1], "k-", label="storage content")
ax2.grid()
ax2.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")

Introduce minimal load, reconfigure heat pump for that, rerun and make plots.

In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

es.add(bus_electricity, bus_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

demand = input_data["Heat load (kW)"][:-1]

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


es.add(electricity_grid, thermal_storage, heat_demand)

In [ ]:
from workshop.model.utilities import load_tespy_cop

tespy_cop = load_tespy_cop()

input_data["simple TESPy COP"] = input_data["Ambient temperature (d°C)"].map(tespy_cop["COP"])

heat_pump = heat_pump_nonconvex = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(
        nominal_value=hp_thermal_power,
        nonconvex=solph.NonConvex(),
        min=0.5,
    )},
    conversion_factors={
        bus_electricity: 1,
        bus_heat_35C: input_data["simple TESPy COP"][:-1],
    },
)

es.add(heat_pump_nonconvex)


model_nonconvex = solph.Model(energysystem=es)
model_nonconvex.solve()


results = result_dict["nonconvex"] = solph.processing.results(model_nonconvex)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(9, 4.5))

ax1.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
ax1.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
ax1.plot(storage_content, "k-", label="storage content")
ax1.set_ylabel("Power (kW) or Energy (kWh)")
ax1.tick_params(axis="x", rotation=50)
ax1.grid()
ax1.legend()

ax2.plot(np.sort(heat_supply)[-2::-1], "r-", label="heat supply")
ax2.plot(np.sort(demand)[::-1], "b--", label="heat demand")
ax2.plot(np.sort(storage_content)[::-1], "k-", label="storage content")
ax2.grid()
ax2.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")

Get part load dependent COP from TESPy, reconfigure heat pump, rerun and make plots.

In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

es.add(bus_electricity, bus_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

demand = input_data["Heat load (kW)"][:-1]

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


es.add(electricity_grid, thermal_storage, heat_demand)

In [ ]:
from workshop.model.utilities import load_tespy_coefficients

tespy_coefficients = load_tespy_coefficients()

slope = input_data["slope"] = input_data["Ambient temperature (d°C)"].map(tespy_coefficients["slope"])
offset = input_data["offset"] = (input_data["Ambient temperature (d°C)"].map(tespy_coefficients["offset"]))/1e3

heat_pump = heat_pump_offset = solph.components.OffsetTransformer(
        label=f"heat pump",
        inputs={bus_electricity: solph.Flow(
            nominal_value=5,
            min=0.0,
            nonconvex=solph.NonConvex(),
        )},
        outputs={bus_heat_35C: solph.Flow(
            nominal_value=hp_thermal_power,
            nonconvex=solph.NonConvex(),
            min=0.5,
        )},
        coefficients=[offset, slope]
)

es.add(heat_pump_offset)


model_offset = solph.Model(energysystem=es)
model_offset.solve()


results = result_dict["nonconvex"] = solph.processing.results(model_offset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(9, 4.5))

ax1.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
ax1.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
ax1.plot(storage_content, "k-", label="storage content")
ax1.set_ylabel("Power (kW) or Energy (kWh)")
ax1.tick_params(axis="x", rotation=50)
ax1.grid()
ax1.legend()

ax2.plot(np.sort(heat_supply)[-2::-1], "r-", label="heat supply")
ax2.plot(np.sort(demand)[::-1], "b--", label="heat demand")
ax2.plot(np.sort(storage_content)[::-1], "k-", label="storage content")
ax2.grid()
ax2.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")